<a href="https://colab.research.google.com/github/khadkechetan/information_extraction/blob/main/Docvqa/FLAN_t5_large_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: **Chetan Khadke**

Email: khadkechetan@gmail.com

# Install the packages

In [1]:
!pip install transformers PyPDF2

# Read the document


- Consider using prefer package for text extraction from images/pdf.

In [3]:
from PyPDF2 import PdfReader

reader = PdfReader("/content/invoice_107_charspace_108.pdf") # Read the uploaded PDF file. Upload your file in collab or read from URL
pdf_text = ""
print("Total pages=",len(reader.pages)) 
page_numbers_to_read = [0] # Specify page number

# Read the given pages
for page in page_numbers_to_read:
    page_text = reader.pages[page].extract_text()
    pdf_text += page_text

Total pages= 1


In [4]:
print(pdf_text)

Invoice no: 82545881
Date of issue:
09/25/2011
Seller:
Campbell, Callahan and Gomez
2969 Todd Orchard Apt. 721
Port James, FL 83598
Tax Id: 958-83-8233
IBAN: GB86WKRJ04578791818338
Client:
Keller-Crosby
280 Kim Valleys Suite 217
Angelaburgh, DE 97356
Tax Id: 941-79-6209
ITEMS
No.
Description
Qty
UM
Net price
Net worth
VAT [%]
Gross
worth
 
Under Armour UA Curry 3
Black/Black Size 6Y
1.
5,00
each
 4,90
 24,50
 10%
 26,95
 
Skechers Hypno Splash Light Up
Shoes Size 4 Medium Big Kid
New
2.
5,00
each
24,99
 124,95
 10%
 137,44
 
Native Charley White
Sandals~Size Infant/Child
6~Boys/Girls~Very Good
Condition
3.
5,00
each
13,00
 65,00
 10%
 71,50
 
Minecraft Creative Stuffed Plush
Kids Childrens Gift Cute Toy
Birthday Christmas
4.
1,00
each
 7,25
 7,25
 10%
 7,98
 
SUMMARY
VAT [%]
 Net worth
 VAT
 Gross worth
 
10%
 221,70
 22,17
 243,87
 
 Total
 $ 221,70
 $ 22,17
 $ 243,87
 


# Load the Model

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large").to(device)

# Inference on Model

In [7]:
def query_from_list(query, options, tok_len):
    t5query = f"""Question: "{query}" Context: {options}"""
    inputs = tokenizer(t5query, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=tok_len)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [12]:
# specify you question here. 
questions = [
    "What is the Invoice no?",
    "What is the Invoice issue Date?",
    "What is the Seller name?",
    "What is the Client name?",
    "What is the Client Address?",
    "What is the Seller Address?",
    "What is the Total Net worth?",
    "What is the total Growth worth amount?"
]

In [9]:
# specify you question here. 
questions = [
    "How much is the Gross worth for Skechers Hypno Splash Light Up Shoes?",
]

In [13]:
%%time
# Call the LLM with input data and instruction
input_data=pdf_text
results = {question:query_from_list(question,  input_data, 30)  for question in questions}

CPU times: user 1min 31s, sys: 0 ns, total: 1min 31s
Wall time: 1min 32s


In [14]:
results

{'What is the Invoice no?': ['82545881'],
 'What is the Invoice issue Date?': ['09/25/2011'],
 'What is the Seller name?': ['Campbell, Callahan and Gomez'],
 'What is the Client name?': ['Keller-Crosby'],
 'What is the Client Address?': ['Keller-Crosby 280 Kim Valleys Suite 217 Angelaburgh, DE 97356'],
 'What is the Seller Address?': ['2969 Todd Orchard Apt. 721 Port James, FL 83598'],
 'What is the Total Net worth?': ['221,70'],
 'What is the total Growth worth amount?': ['243,87']}